In [5]:
"""
AI Newsletter Crew – Hierarchical Multi-Agent Architecture
----------------------------------------------------------

This script demonstrates a *multi-agent workflow* using CrewAI where:
- A Manager LLM oversees everything (hierarchical mode)
- Multiple Agents handle different steps: fetching, analyzing, compiling news
- A callback saves the final newsletter to a markdown file

This version includes:
✔ Tight, safe, structured prompts  
✔ Clear agent goals  
✔ Naive-friendly comments  
✔ Strong anti-hallucination rules  
✔ Clean workflow

Architecture:
    Manager LLM
         │
         ▼
      Editor  (supervisor agent)
         │
         ├──► NewsFetcher
         ├──► NewsAnalyzer
         └──► NewsletterCompiler

"""

# ---------------------------- IMPORTS ----------------------------
from crewai import Crew, Process, LLM, Agent, Task
from langchain_openai import ChatOpenAI
from crewai_tools import SerperDevTool
from dotenv import load_dotenv
from datetime import datetime
import os
load_dotenv()  # Load environment variables from .env


# ---------------------------- MODEL SETUP ----------------------------
# CrewAI wrapper around an OpenAI model
llm = LLM(model="gpt-4o-mini")
 
# Manager LLM (boss of the crew)
OpenAIGPT4 = ChatOpenAI(model="gpt-4o-mini")

# Web search tool for agents who need it
search_tool = SerperDevTool()


# ---------------------------- SAVE OUTPUT ----------------------------
def save_markdown(task_output):
    """
    Save the final newsletter to a timestamped markdown file.
    This captures the safest available text field from the TaskOutput.
    """
    content = getattr(task_output, "output_text", None) or \
              getattr(task_output, "raw_output", str(task_output))

    filename = f"newsletter_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"

    with open(filename, "w", encoding="utf-8") as f:
        f.write(content)

    print(f"✅ Newsletter saved as {filename}")



# ===================================================================
# ████ 1. DEFINE AGENTS (specialized workers with clear goals) ████
# ===================================================================

# ---------------------------- EDITOR ----------------------------
editor = Agent(
    role='Editor',
    goal="""Ensure the newsletter is accurate, consistent in tone,
    logically structured, and follows professional editorial standards.
    Enforce formatting rules and request corrections from other agents if needed.""",
    backstory="""You are a senior editorial guardian with strong commitment
    to clarity, accuracy, neutrality, and clean formatting. You never allow
    sloppy structure, missing sections, or hallucinated claims.""",
    allow_delegation=True,
    verbose=True,
    llm=llm,
    max_iter=8,
)

# ---------------------------- NEWS FETCHER ----------------------------
news_fetcher = Agent(
    role='NewsFetcher',
    goal="""Retrieve only real, verifiable AI news stories from the last 24 hours,
    ensuring accuracy and avoiding guesses. Provide reliable URLs and short summaries.""",
    backstory="""You are a reliable, detail-oriented digital investigator who
    searches the web responsibly and never invents news.""",
    tools=[search_tool],
    allow_delegation=True,
    verbose=True,
    llm=llm
)

# ---------------------------- NEWS ANALYZER ----------------------------
news_analyzer = Agent(
    role='NewsAnalyzer',
    goal="""Convert raw news items into clear, structured markdown-formatted
    analysis articles. Each article must include:
    - A headline
    - A 'Rundown' paragraph
    - Bullet-point details
    - A 'Why it matters' section
    - No hallucinations or invented claims""",
    backstory="""You are a precise analyst who simplifies complex news without
    losing meaning, detects key insights, and avoids speculation.""",
    tools=[search_tool],
    allow_delegation=True,
    verbose=True,
    llm=llm
)

# ---------------------------- NEWSLETTER COMPILER ----------------------------
newsletter_compiler = Agent(
    role='NewsletterCompiler',
    goal="""Assemble all analyzed articles into a polished, engaging newsletter
    in markdown format with consistent formatting, spacing, headings, and style.""",
    backstory="""You are a meticulous formatter who ensures the final newsletter
    reads like a premium publication: clean, organized, visually scannable.""",
    allow_delegation=False,
    verbose=True,
    llm=llm
)



# ===================================================================
# ████ 2. DEFINE TASKS (what each agent must deliver) ████
# ===================================================================

# ---------------------------- TASK 1 – FETCH NEWS ----------------------------
fetch_news_task = Task(
    description=f"""
    Fetch the top *real* AI news stories from the last 24 hours.
    STRICT RULES:
    - Use the search tool.
    - Only include verifiable real articles.
    - Provide: title, URL, 2–3 sentence summary.
    - Avoid predictions, rumors, or invented news.
    """,
    agent=news_fetcher,
    async_execution=True,
    expected_output="""
    A JSON-style list of at least 5 AI news stories:
    [
      {
        "title": "...",
        "url": "...",
        "summary": "..."
      },
      ...
    ]
    """
)

# ---------------------------- TASK 2 – ANALYZE NEWS ----------------------------
analyze_news_task = Task(
    description="""
    Analyze each fetched news story and convert it into a markdown article.
    Each article MUST follow this exact structure:
    ## {Headline}

    **The Rundown:** A 2–3 sentence high-level explanation.

    **The Details:**
    - Bullet point
    - Bullet point
    - Bullet point

    **Why it matters:** A brief explanation of the real-world significance.

    RULES:
    - Minimum 5 articles.
    - No hallucinations.
    - No invented facts.
    - Stick closely to the original source content.
    """,
    agent=news_analyzer,
    async_execution=True,
    expected_output="At least 5 markdown-formatted news analysis articles."
)

# ---------------------------- TASK 3 – COMPILE NEWSLETTER ----------------------------
compile_newsletter_task = Task(
    description="""
    Combine all analyzed articles into a single, professionally structured
    markdown newsletter.

    REQUIRED STRUCTURE:
        '# Top stories in AI today:\\n\\n
        - AI takes spotlight in Super Bowl commercials\\n
        - company ABC seeks TRILLIONS for global AI chip initiative\\n\\n

        ## AI takes spotlight in Super Bowl commercials\\n\\n
        **The Rundown:** AI made a splash in this year\'s Super Bowl commercials...\\n\\n
        **The details:**...\\n\\n
        **Why it matters::**...\\n\\n
        ## XYZ seeks TRILLIONS for global AI chip initiative\\n\\n
        **The Rundown:** ABC CEO is reportedly angling to raise TRILLIONS of dollars...\\n\\n'
        **The details:**...\\n\\n
        **Why it matters::**...\\n\\n

    RULES:
    - Maintain consistent spacing and markdown styling.
    - No missing sections.
    - No duplicated content.
    - No hallucinated additions.
    """,
    agent=newsletter_compiler,
    expected_output="A complete polished AI newsletter in markdown format.",
    callback=save_markdown
)



# ===================================================================
# ████ 3. FORM THE CREW (supervisor + agents + tasks) ████
# ===================================================================

crew = Crew(
    agents=[editor, news_fetcher, news_analyzer, newsletter_compiler],
    tasks=[fetch_news_task, analyze_news_task, compile_newsletter_task],
    process=Process.hierarchical,  # Manager delegates tasks like a boss
    manager_llm=OpenAIGPT4,
    verbose=True
)



# ===================================================================
# ████ 4. RUN THE WORKFLOW ████
# ===================================================================

results = crew.kickoff()

print("\n=== Crew Work Results ===\n")
print(results)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 997830c1-bff7-4c55-91d5-96060a3b2f59                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Fetch the top *real* AI news stories from the last 24 hours.                                               │
│      STRICT RULES:                                                                                              │
│      - Use the search tool.                                                                                     │
│      - Only include verifiable real articles.                                                                   │
│      - Provide: title, URL, 2–3 sentence summary.                                                               │
│      - Avoid predictions, rumors, or invented news.                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Analyze each fetched news story and convert it into a markdown article.                                    │
│      Each article MUST follow this exact structure:                                                             │
│      ## {Headline}                                                                                              │
│                                                                                                                 │
│      **The Rundown:** A 2–3 sentence high-level explanation.                                                    │
│                                                                                                                 │
│      **The Details:**                                                                                           │
│      - Bullet point                                                                                             │
│      - Bullet point                                                                                             │
│      - Bullet point                                                                                             │
│                                                                                                                 │
│      **Why it matters:** A brief explanation of the real-world significance.                                    │
│                                                                                                                 │
│      RULES:                                                                                                     │
│      - Minimum 5 articles.                                                                                      │
│      - No hallucinations.                                                                                       │
│      - No invented facts.                                                                                       │
│      - Stick closely to the original source content.                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Top stories in AI today:                                                                                     │
│                                                                                                                 │
│  - AI takes spotlight in Super Bowl commercials                                                                 │
│  - ABC seeks TRILLIONS for global AI chip initiative                                                            │
│                                                                                                                 │
│  ## AI takes spotlight in Super Bowl commercials                                                                │
│                                                                                                                 │
│  **The Rundown:** AI made a splash in this year's Super Bowl commercials, highlighting its growing influence    │
│  in mainstream media and advertising.                                                                           │
│                                                                                                                 │
│  **The details:** Several companies showcased innovative AI technologies in their ads, emphasizing how AI is    │
│  transforming industries and captivating audiences. These commercials demonstrated AI applications ranging      │
│  from personalized shopping experiences to revolutionizing customer service.                                    │
│                                                                                                                 │
│  **Why it matters:** The integration of AI in such a high-profile event like the Super Bowl signifies the       │
│  technology's acceptance within major consumer markets, setting a precedent for future marketing strategies     │
│  that leverage AI.                                                                                              │
│                                                                                                                 │
│  ## ABC seeks TRILLIONS for global AI chip initiative                                                           │
│                                                                                                                 │
│  **The Rundown:** ABC CEO is reportedly angling to raise TRILLIONS of dollars to develop a next-generation AI   │
│  chip that promises to accelerate AI computation significantly.                                                 │
│                                                                                                                 │
│  **The details:** The proposed AI chip is expected to enhance processing speed and efficiency, enabling         │
│  advanced machine learning models and AI applications. This initiative could potentially lead to breakthroughs  │
│  in various sectors including healthcare, automotive, and robotics.                                             │
│                                                                                                                 │
│  **Why it matters:** This initiative underscores the importance of hardware advancements in the AI sector.      │
│  Significant investments in AI infrastructure can accelerate innovation and maintain competitiveness in the     │
│  rapidly evolving technology landscape.                

✅ Newsletter saved as newsletter_20251122_143629.md


╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f600432a-70d1-4446-8922-3af861c0349d                                                                     │
│  Agent: Crew Manager                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 997830c1-bff7-4c55-91d5-96060a3b2f59                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Top stories in AI today:                                                                       │
│                                                                                                                 │
│  - AI takes spotlight in Super Bowl commercials                                                                 │
│  - ABC seeks TRILLIONS for global AI chip initiative                                                            │
│                                                                                                                 │
│  ## AI takes spotlight in Super Bowl commercials                                                                │
│                                                                                                                 │
│  **The Rundown:** AI made a splash in this year's Super Bowl commercials, highlighting its growing influence    │
│  in mainstream media and advertising.                                                                           │
│                                                                                                                 │
│  **The details:** Several companies showcased innovative AI technologies in their ads, emphasizing how AI is    │
│  transforming industries and captivating audiences. These commercials demonstrated AI applications ranging      │
│  from personalized shopping experiences to revolutionizing customer service.                                    │
│                                                                                                                 │
│  **Why it matters:** The integration of AI in such a high-profile event like the Super Bowl signifies the       │
│  technology's acceptance within major consumer markets, setting a precedent for future marketing strategies     │
│  that leverage AI.                                                                                              │
│                                                                                                                 │
│  ## ABC seeks TRILLIONS for global AI chip initiative                                                           │
│                                                                                                                 │
│  **The Rundown:** ABC CEO is reportedly angling to raise TRILLIONS of dollars to develop a next-generation AI   │
│  chip that promises to accelerate AI computation significantly.                                                 │
│                                                                                                                 │
│  **The details:** The proposed AI chip is expected to enhance processing speed and efficiency, enabling         │
│  advanced machine learning models and AI applications. This initiative could potentially lead to breakthroughs  │
│  in various sectors including healthcare, automotive, and robotics.                                             │
│                                                                                                                 │
│  **Why it matters:** This initiative underscores the importance of hardware advancements in the AI sector.      │
│  Significant investments in AI infrastructure can acce


=== Crew Work Results ===

# Top stories in AI today:

- AI takes spotlight in Super Bowl commercials
- ABC seeks TRILLIONS for global AI chip initiative

## AI takes spotlight in Super Bowl commercials

**The Rundown:** AI made a splash in this year's Super Bowl commercials, highlighting its growing influence in mainstream media and advertising. 

**The details:** Several companies showcased innovative AI technologies in their ads, emphasizing how AI is transforming industries and captivating audiences. These commercials demonstrated AI applications ranging from personalized shopping experiences to revolutionizing customer service.

**Why it matters:** The integration of AI in such a high-profile event like the Super Bowl signifies the technology's acceptance within major consumer markets, setting a precedent for future marketing strategies that leverage AI.

## ABC seeks TRILLIONS for global AI chip initiative

**The Rundown:** ABC CEO is reportedly angling to raise TRILLIONS of dol